In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# MetabolicAI: Live Demo Notebook\n",
    "\n",
    "Interact with your running API — log new entries and see your TDEE update instantly.\n",
    "\n",
    "> _Ensure your FastAPI app is running on http://localhost:8000 before starting!_\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Setup ----\n",
    "import requests\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "BASE_URL = \"http://localhost:8000\"\n",
    "API_KEY = \"changeme\"  # Change if you use a different key!\n",
    "USER_ID = \"notebookdemo\"\n",
    "HEADERS = {\n",
    "    \"X-API-Key\": API_KEY,\n",
    "    \"X-User-Id\": USER_ID\n",
    "}\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Register or update your profile ----\n",
    "profile = {\n",
    "    \"user_id\": USER_ID,\n",
    "    \"age\": 27,\n",
    "    \"gender\": \"male\",\n",
    "    \"height_cm\": 180,\n",
    "    \"body_fat_pct\": 18,\n",
    "    \"current_weight\": 76\n",
    "}\n",
    "try:\n",
    "    r = requests.post(f\"{BASE_URL}/user\", json=profile, headers={\"X-API-Key\": API_KEY}, timeout=5)\n",
    "    r.raise_for_status()\n",
    "    print(\"Profile created/updated:\", r.json())\n",
    "except Exception as e:\n",
    "    print(\"Could not create/update profile:\", e)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Function: Log a new entry & get current TDEE, with error handling ----\n",
    "def log_entry(date, weight=None, calories=None):\n",
    "    payload = {\"date\": date}\n",
    "    if weight is not None:\n",
    "        payload[\"weight\"] = weight\n",
    "    if calories is not None:\n",
    "        payload[\"calories\"] = calories\n",
    "    try:\n",
    "        r = requests.post(f\"{BASE_URL}/entry\", json=payload, headers=HEADERS, timeout=5)\n",
    "        r.raise_for_status()\n",
    "        print(\"Entry logged:\", r.json())\n",
    "    except Exception as e:\n",
    "        print(f\"Error logging entry: {e}\")\n",
    "        return\n",
    "\n",
    "    # Get updated prediction\n",
    "    try:\n",
    "        tdee_resp = requests.get(f\"{BASE_URL}/tdee\", headers=HEADERS, timeout=5)\n",
    "        if tdee_resp.status_code == 200:\n",
    "            tdee = tdee_resp.json()[\"tdee\"]\n",
    "            print(f\"Updated TDEE prediction: {tdee} kcal\")\n",
    "        else:\n",
    "            print(\"Not enough data for prediction yet.\")\n",
    "    except Exception as e:\n",
    "        print(f\"Error fetching TDEE: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Example: Add new entries ----\n",
    "log_entry(\"2025-07-14\", weight=76, calories=2200)\n",
    "log_entry(\"2025-07-15\", weight=75.8, calories=2180)\n",
    "log_entry(\"2025-07-16\", weight=75.7, calories=2150)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Get all entries as a DataFrame ----\n",
    "def get_history():\n",
    "    try:\n",
    "        r = requests.get(f\"{BASE_URL}/history\", headers=HEADERS, timeout=5)\n",
    "        r.raise_for_status()\n",
    "        df = pd.DataFrame(r.json()[\"entries\"])\n",
    "        return df\n",
    "    except Exception as e:\n",
    "        print(\"Error fetching history:\", e)\n",
    "        return pd.DataFrame()\n",
    "\n",
    "df = get_history()\n",
    "df"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Visualize weight trend ----\n",
    "if not df.empty and \"weight\" in df:\n",
    "    df[\"date\"] = pd.to_datetime(df[\"date\"])\n",
    "    df = df.sort_values(\"date\")\n",
    "    plt.plot(df[\"date\"], df[\"weight\"], marker=\"o\")\n",
    "    plt.title(\"Weight Progress\")\n",
    "    plt.xlabel(\"Date\")\n",
    "    plt.ylabel(\"Weight (kg)\")\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"No entries with weight to plot yet.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Plot TDEE Prediction Over Time ----\n",
    "def plot_tdee_over_time():\n",
    "    df = get_history()\n",
    "    if df.empty or \"date\" not in df:\n",
    "        print(\"No entries yet.\")\n",
    "        return\n",
    "    tdee_dates = []\n",
    "    tdees = []\n",
    "    for date in sorted(df['date'].unique()):\n",
    "        try:\n",
    "            temp_payload = {\"date\": date}\n",
    "            row = df[df['date'] == date].iloc[0]\n",
    "            if pd.notna(row.get(\"weight\")):\n",
    "                temp_payload[\"weight\"] = float(row[\"weight\"])\n",
    "            if pd.notna(row.get(\"calories\")):\n",
    "                temp_payload[\"calories\"] = int(row[\"calories\"])\n",
    "            # Log the entry again (harmless, upserts)\n",
    "            requests.post(f\"{BASE_URL}/entry\", json=temp_payload, headers=HEADERS)\n",
    "            tdee_resp = requests.get(f\"{BASE_URL}/tdee\", headers=HEADERS)\n",
    "            if tdee_resp.status_code == 200:\n",
    "                tdee_dates.append(date)\n",
    "                tdees.append(tdee_resp.json()[\"tdee\"])\n",
    "        except Exception:\n",
    "            continue\n",
    "    if tdee_dates and tdees:\n",
    "        plt.figure(figsize=(7, 4))\n",
    "        plt.plot(pd.to_datetime(tdee_dates), tdees, marker=\"o\", color=\"orange\")\n",
    "        plt.title(\"TDEE Prediction Over Time\")\n",
    "        plt.xlabel(\"Date\")\n",
    "        plt.ylabel(\"TDEE (kcal)\")\n",
    "        plt.grid(True)\n",
    "        plt.show()\n",
    "    else:\n",
    "        print(\"Not enough data to plot TDEE history yet.\")\n",
    "\n",
    "# Example usage:\n",
    "plot_tdee_over_time()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ---- Progress Summary ----\n",
    "def progress_summary():\n",
    "    df = get_history()\n",
    "    if df.empty:\n",
    "        print(\"No data logged yet.\")\n",
    "        return\n",
    "    print(f\"Entries logged: {len(df)}\")\n",
    "    if \"weight\" in df and df[\"weight\"].notna().sum() > 1:\n",
    "        start = df[\"weight\"].dropna().iloc[0]\n",
    "        end = df[\"weight\"].dropna().iloc[-1]\n",
    "        print(f\"Weight change: {start:.1f}kg → {end:.1f}kg ({end-start:+.1f}kg)\")\n",
    "    if \"calories\" in df and df[\"calories\"].notna().sum() > 0:\n",
    "        print(f\"Avg daily calories: {df['calories'].dropna().mean():.0f}\")\n",
    "    try:\n",
    "        tdee = requests.get(f\"{BASE_URL}/tdee\", headers=HEADERS, timeout=5)\n",
    "        if tdee.status_code == 200:\n",
    "            print(\"Current TDEE prediction:\", tdee.json()[\"tdee\"])\n",
    "    except Exception as e:\n",
    "        print(f\"TDEE fetch error: {e}\")\n",
    "\n",
    "# Example usage\n",
    "progress_summary()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
